In [3]:
!pip install transformers Sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00


# Upload dataset

Without preprocessing!

In [4]:
path_to_zip_file = '/content/20230504_20230614.zip'
directory_to_extract_to = 'dataset'

In [5]:
import zipfile


with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

In [6]:
import pandas as pd

posts = pd.read_json('/content/dataset/20230504_20230614/posts.json')
comments = pd.read_json('/content/dataset/20230504_20230614/comments.json')

In [7]:
posts.columns

Index(['text', 'id', 'owner_id', 'has_comments', 'updated_at', 'created_at',
       'is_deleted'],
      dtype='object')

In [8]:
posts

,text,id,owner_id,has_comments,updated_at,created_at,is_deleted
0,Мой Любимый каратист 🥰,9224,27263088,False,2023-05-14 01:00:27,2023-05-13 18:37:26,False
1,14,9770,46250906,False,2023-05-14 01:00:27,2023-05-13 16:47:17,False
2,17,9769,46250906,False,2023-05-14 01:00:27,2023-05-13 16:47:00,False
3,13,9768,46250906,False,2023-05-14 01:00:27,2023-05-13 16:46:06,False
4,31!!,9767,46250906,False,2023-05-14 01:00:27,2023-05-13 16:45:34,False
...,...,...,...,...,...,...,...
287537,,807,38164175,False,2023-06-14 20:04:09,2023-06-14 08:41:49,False
287538,❗❗❗СТОП 15 ИЮНЯ ❗❗❗ \n🎒👛👜СУМКИНЫ ДЕТИ👜👛🎒 \n🔥💞🔥...,34701,38204717,False,2023-06-14 20:04:09,2023-06-14 08:30:00,False
287539,890 ОПТ \nКроссовки женские (натуральная кожа)...,34700,38204717,False,2023-06-14 20:04:09,2023-06-14 08:02:00,False
287540,❗❗❗СТОП 15 ИЮНЯ ❗❗❗ \n🎒👛👜СУМКИНЫ ДЕТИ👜👛🎒 \n🔥💞🔥...,34699,38204717,False,2023-06-14 20:04:09,2023-06-14 07:30:00,False


# cointegrated/rubert-tiny-sentiment-balanced

In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

In [15]:
y_preds = []

for xx in posts.iloc[:10000]['text']:
  pred = get_sentiment(xx)
  y_preds.append(pred)

In [16]:
df_posts = posts.iloc[:10000]
df_posts['cointegrated/rubert'] = y_preds
df_posts.head()

<ipython-input-16-5355d2f1eb17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_posts['cointegrated/rubert'] = y_preds


,text,id,owner_id,has_comments,updated_at,created_at,is_deleted,cointegrated/rubert
0,Мой Любимый каратист 🥰,9224,27263088,False,2023-05-14 01:00:27,2023-05-13 18:37:26,False,positive
1,14,9770,46250906,False,2023-05-14 01:00:27,2023-05-13 16:47:17,False,neutral
2,17,9769,46250906,False,2023-05-14 01:00:27,2023-05-13 16:47:00,False,neutral
3,13,9768,46250906,False,2023-05-14 01:00:27,2023-05-13 16:46:06,False,neutral
4,31!!,9767,46250906,False,2023-05-14 01:00:27,2023-05-13 16:45:34,False,neutral


# sismetanin/mbart_ru_sum_gazeta-ru-sentiment-rusentiment

In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("sismetanin/mbart_ru_sum_gazeta-ru-sentiment-rusentiment")

model = AutoModelForSequenceClassification.from_pretrained("sismetanin/mbart_ru_sum_gazeta-ru-sentiment-rusentiment")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [19]:
from tqdm import tqdm

y_preds1 = []

for xx in tqdm(posts.iloc[:10000]['text']):
  if len(xx) < 2000:
    preds = classifier(xx)[0]['label']
  else:
    xx = xx[:2000]
    preds = classifier(xx)[0]['label']


  y_preds1.append(preds)

100%|██████████| 10000/10000 [1:13:47<00:00,  2.26it/s]


In [20]:
def encode(label):
  d = {'anger': 'negative',
       'fear': 'negative',
       'joy': 'positive',
       'no_emotion': 'neutral',
       'sadness': 'negative',
       'surprise': 'positive'}

  return d[label]

y_preds1 = [encode(yy) for yy in y_preds1]

KeyError: ignored

In [21]:
y_preds1 = [predicted.lower() for predicted in y_preds1]
df_posts['mbart_ru_sum'] = y_preds1
df_posts.head()

<ipython-input-21-6908d826140c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_posts['mbart_ru_sum'] = y_preds1


,text,id,owner_id,has_comments,updated_at,created_at,is_deleted,cointegrated/rubert,mbart_ru_sum
0,Мой Любимый каратист 🥰,9224,27263088,False,2023-05-14 01:00:27,2023-05-13 18:37:26,False,positive,label_2
1,14,9770,46250906,False,2023-05-14 01:00:27,2023-05-13 16:47:17,False,neutral,label_1
2,17,9769,46250906,False,2023-05-14 01:00:27,2023-05-13 16:47:00,False,neutral,label_1
3,13,9768,46250906,False,2023-05-14 01:00:27,2023-05-13 16:46:06,False,neutral,label_1
4,31!!,9767,46250906,False,2023-05-14 01:00:27,2023-05-13 16:45:34,False,neutral,label_1


In [22]:
set(df_posts['mbart_ru_sum'])

{'label_0', 'label_1', 'label_2', 'label_3', 'label_4'}

In [24]:
df_posts.to_csv('issa_test.csv', index=False)

# blanchefort/rubert-base-cased-sentiment-rusentiment

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("blanchefort/rubert-base-cased-sentiment-rusentiment")

model = AutoModelForSequenceClassification.from_pretrained("blanchefort/rubert-base-cased-sentiment-rusentiment")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
from tqdm import tqdm

y_preds2 = []

for xx in tqdm(posts.iloc[:10000]['text']):
  if len(xx) < 2000:
    preds = classifier(xx)[0]['label']
  else:
    xx = xx[:2000]
    preds = classifier(xx)[0]['label']


  y_preds2.append(preds)

In [ ]:
y_preds1 = [predicted.lower() for predicted in y_preds1]
df_posts['mbart_ru_sum'] = y_preds1
df_posts.head()

# Analyzing dataset

In [9]:
posts[posts['has_comments'] == True]

,text,id,owner_id,has_comments,updated_at,created_at,is_deleted
17,"Сегодня я провожала сильных духом,смелых,увере...",2454,20458770,True,2023-05-14 01:00:27,2023-05-13 07:35:49,False
30,"Любимый муж, поздравляю нас с тобой со стально...",7881,56105976,True,2023-05-14 01:00:27,2023-05-12 14:06:09,False
57,#сместа_в_карьеру\n#РоссийскоеОбществоЗнание\n...,7875,56105976,True,2023-05-14 01:00:27,2023-05-11 06:53:07,False
91,Мы Помним! Мы Гордимся!,7872,56105976,True,2023-05-14 01:00:27,2023-05-09 10:03:02,False
114,"Живите так, чтоб ангелы остались без работы, а...",9565,14939751,True,2023-05-14 01:00:27,2023-05-08 09:34:09,False
...,...,...,...,...,...,...,...
287517,"Комплект трикотажный для девочек: кардиган, юб...",4167,37809445,True,2023-06-14 20:04:09,2023-06-14 12:26:46,False
287518,Российский размер: 158; 164; 152\nНОВАЯ ЦЕНА 1...,4165,37809445,True,2023-06-14 20:04:09,2023-06-14 12:08:27,False
287532,КОСТЮМ СПОРТИВНЫЙ Рост: 152; 164; 158\nНОВАЯ Ц...,4162,37809445,True,2023-06-14 20:04:09,2023-06-14 09:01:42,False
287533,Российский размер: 92; 110; 86; 98; 104\nНОВАЯ...,4161,37809445,True,2023-06-14 20:04:09,2023-06-14 08:55:12,False


In [10]:
posts.iloc[17]

text            Сегодня я провожала сильных духом,смелых,увере...
id                                                           2454
owner_id                                                 20458770
has_comments                                                 True
updated_at                                    2023-05-14 01:00:27
created_at                                    2023-05-13 07:35:49
is_deleted                                                  False
Name: 17, dtype: object

In [11]:
for i in posts[posts['has_comments'] == True].index.to_list():

  print(comments[comments['post_id'] == posts.iloc[i]['id']])

Выходные данные были обрезаны до нескольких последних строк (5000).
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, saved_at]
Index: []
Empty DataFrame
Columns: [post_id, author_id, text, id, created_at, 

KeyboardInterrupt: ignored

In [ ]:
posts[posts['id'] == '5265p122093213']